# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql("SELECT * FROM disaster_info", engine)
cols = df.columns.tolist()[4:]
# X = df[cols]
# Y = df['genre']
X = df['message']


In [3]:
df[cols].head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df.shape

(26207, 40)

In [5]:
df = df.drop_duplicates()

In [6]:
df[cols].max()

related                   2.0
request                   1.0
offer                     1.0
aid_related               1.0
medical_help              1.0
medical_products          1.0
search_and_rescue         1.0
security                  1.0
military                  1.0
child_alone               0.0
water                     1.0
food                      1.0
shelter                   1.0
clothing                  1.0
money                     1.0
missing_people            1.0
refugees                  1.0
death                     1.0
other_aid                 1.0
infrastructure_related    1.0
transport                 1.0
buildings                 1.0
electricity               1.0
tools                     1.0
hospitals                 1.0
shops                     1.0
aid_centers               1.0
other_infrastructure      1.0
weather_related           1.0
floods                    1.0
storm                     1.0
fire                      1.0
earthquake                1.0
cold      

In [7]:
df.loc[df['related'] == 2, 'related'] = 1


In [8]:
Y = df[cols]
Y

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [9]:
y = Y.values

In [10]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [12]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2. Write a tokenization function to process your text data

In [13]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
   
    clean_tokens = [w for w in clean_tokens if w not in stopwords.words("english")]
    return clean_tokens
    pass

In [14]:
def display_results(Y_test, Y_pred):
    labels = np.unique(Y_pred)
    confusion_mat = confusion_matrix(Y_test, Y_pred, labels=labels)
    accuracy = (Y_pred == Y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [21]:
random_state = 42
def model_pipeline():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
        ])),

        ('clf', RandomForestClassifier())
    ])

    return pipeline


In [22]:
import numpy as np
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [23]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=random_state)


model = model_pipeline()

# train classifier
model.fit(X_train, Y_train)

# predict on test data
Y_pred = model.predict(X_test)

# display results
# display_results(Y_test, Y_pred)



In [14]:
Y

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]])

In [15]:
Y_test

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  1.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]])

In [16]:
Y_pred

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]])

In [17]:
accuracy = (Y_pred == Y_test).mean()
accuracy

0.92442451990334473

In [18]:
len(cols)

36

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [19]:
Y_test.shape[1]

36

In [20]:
from sklearn.metrics import classification_report
for c in range(Y_test.shape[1]):
    y_true = Y_test[c]
    y_pred = Y_pred[c]
    print(cols[c], classification_report(y_true, y_pred))

related              precision    recall  f1-score   support

        0.0       1.00      0.97      0.99        36
        1.0       0.00      0.00      0.00         0

avg / total       1.00      0.97      0.99        36

request              precision    recall  f1-score   support

        0.0       1.00      0.97      0.99        36
        1.0       0.00      0.00      0.00         0

avg / total       1.00      0.97      0.99        36

offer              precision    recall  f1-score   support

        0.0       1.00      1.00      1.00        35
        1.0       1.00      1.00      1.00         1

avg / total       1.00      1.00      1.00        36

aid_related              precision    recall  f1-score   support

        0.0       1.00      1.00      1.00        36

avg / total       1.00      1.00      1.00        36

medical_help              precision    recall  f1-score   support

        0.0       0.81      1.00      0.89        29
        1.0       0.00      0.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [21]:
from sklearn.model_selection import GridSearchCV
parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'clf__n_estimators': [100],
        'clf__min_samples_split': [2, 3, 4]
    }

cv = GridSearchCV(model, param_grid=parameters)

In [69]:
from sklearn.model_selection import GridSearchCV
parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'clf__n_estimators': [10, 20],
        'clf__min_samples_split': [2, 3, 4]
    }

cv = GridSearchCV(model, param_grid=parameters)

In [ ]:
cv.fit(X_train, Y_train)
Y_pred_1 = cv.predict(X_test)

In [23]:
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",cv.best_estimator_)
print("\n The best estimator's score across ALL searched params:\n", cv.best_estimator_.score)
print("\n The best score across ALL searched params:\n",cv.best_score_)
print("\n The best parameters across ALL searched params:\n", cv.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:
 Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

 The best estimator's score across ALL searched params:
 <function Pipeline.score at 0x7f1dd6847f28>

 The best score across ALL searched params:
 0.196412996075

 The best parameters across ALL searched params:
 {'clf__min_samples_split': 3, 'clf__n_estimators': 100, 'features__text_pipeline__vect__ngram_range': (1, 2)}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [24]:
for c in range(Y_test.shape[1]):
    y_true = Y_test[c]
    y_pred = Y_pred_1[c]
    print(cols[c], classification_report(y_true, y_pred))

related              precision    recall  f1-score   support

        0.0       1.00      0.97      0.99        36
        1.0       0.00      0.00      0.00         0

avg / total       1.00      0.97      0.99        36

request              precision    recall  f1-score   support

        0.0       1.00      0.97      0.99        36
        1.0       0.00      0.00      0.00         0

avg / total       1.00      0.97      0.99        36

offer              precision    recall  f1-score   support

        0.0       1.00      1.00      1.00        35
        1.0       1.00      1.00      1.00         1

avg / total       1.00      1.00      1.00        36

aid_related              precision    recall  f1-score   support

        0.0       1.00      0.97      0.99        36
        1.0       0.00      0.00      0.00         0

avg / total       1.00      0.97      0.99        36

medical_help              precision    recall  f1-score   support

        0.0       0.83      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [17]:
from sklearn.feature_extraction.text import HashingVectorizer

In [24]:
random_state = 42
def model_pipeline_improved():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('hvect', HashingVectorizer(tokenizer=tokenize)),
            ])),

#             ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', RandomForestClassifier())
    ])

    return pipeline


In [ ]:
model_improved = model_pipeline_improved()

# train classifier
model_improved.fit(X_train, Y_train)

# predict on test data
Y_pred_improved = model_improved.predict(X_test)


In [27]:
for c in range(Y_test.shape[1]):
    y_true = Y_test[c]
    y_pred = Y_pred_improved[c]
    print(cols[c], classification_report(y_true, y_pred))

related              precision    recall  f1-score   support

        0.0       1.00      0.92      0.96        36
        1.0       0.00      0.00      0.00         0

avg / total       1.00      0.92      0.96        36

request              precision    recall  f1-score   support

        0.0       1.00      1.00      1.00        36

avg / total       1.00      1.00      1.00        36

offer              precision    recall  f1-score   support

        0.0       1.00      0.97      0.99        35
        1.0       0.50      1.00      0.67         1

avg / total       0.99      0.97      0.98        36

aid_related              precision    recall  f1-score   support

        0.0       1.00      0.97      0.99        36
        1.0       0.00      0.00      0.00         0

avg / total       1.00      0.97      0.99        36

medical_help              precision    recall  f1-score   support

        0.0       0.83      1.00      0.91        29
        1.0       1.00      0.14      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [28]:
# loading library
import pickle
# create an iterator object with write permission - model.pkl
with open('model_pkl', 'wb') as files:
    pickle.dump(model, files)

In [29]:
# loading library
import pickle
# create an iterator object with write permission - model.pkl
with open('model_improved_pkl', 'wb') as files:
    pickle.dump(model_improved, files)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [1]:
# import packages
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import HashingVectorizer
import pickle
import sys
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def load_data(data_file):
    # read in file
    engine = create_engine('sqlite:///InsertDatabaseName.db')
    df = pd.read_sql(data_file, engine)

    # clean data
    df.drop_duplicates()

    # load to database
    engine = create_engine('sqlite:///InsertDatabaseName.db')
    df.to_sql('DisasterResponse', engine,if_exists = 'replace', index=False) 
    
    # define features and label arrays
    cols = df.columns.tolist()[4:]
    X = df['message']
    y = df[cols]
    X = X.values
    y = y.values
        
    return X, y, cols


In [4]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
   
    clean_tokens = [w for w in clean_tokens if w not in stopwords.words("english")]
    return clean_tokens

In [25]:
def display_results(y_test, y_pred, cols):
    # output model test results
    for c in range(y_test.shape[1]):
        print(c)
        y_true = y_test[c]
        y_pr = y_pred[c]
        precision, recall, fscore, support = precision_recall_fscore_support(y_true, y_pr, average='weighted')
        print('\nReport for the column ({}):\n'.format(cols[c]))
        print('Precision: {}'.format(round(precision, 2)))
        print('Recall: {}'.format(round(recall, 2)))
        print('F-score: {}'.format(round(fscore, 2)))


In [30]:
def build_model():
    # text processing and model pipeline

    model = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
        ])),

        ('clf', RandomForestClassifier())
    ])
    
    
# define parameters for GridSearchCV
    parameters = {
#         'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'clf__n_estimators': [10, 30],
        'clf__min_samples_split': [2, 3]
    }

    # create gridsearch object and return as final model pipeline
    model_pipeline = GridSearchCV(model, param_grid=parameters)
    
    return model_pipeline

In [34]:
def build_model():
    # text processing and model pipeline

    model = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    
# define parameters for GridSearchCV
    parameters = {'vect__min_df': [1, 3],
                'tfidf__use_idf': [True, False],
                'clf__estimator__n_estimators': [10, 25],
                'clf__estimator__min_samples_split': [2, 4]
                 }

    # create gridsearch object and return as final model pipeline
    model_pipeline = GridSearchCV(model, param_grid=parameters)
    
    return model_pipeline

In [35]:
def train(X, y, model):
    
    # train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    # train classifier
    model = build_model()
    model.fit(X_train, y_train)

    # predict on test data
    y_pred = model.predict(X_test)

    return model, y_test, y_pred

In [16]:
def export_model(model):
    # Export model as a pickle file
    # create an iterator object with write permission - model.pkl
    with open('model_pkl', 'wb') as files:
        pickle.dump(model, files)

In [36]:
def main():
#     data_file = sys.argv[1]
    data_file = 'disaster_info'

    print('Loading data...\n')
    X, y, cols = load_data(data_file)
        
    print('Building model...\n')
    model = build_model()
              
    print('Building test and train data, training model...\n')
    model, y_test, y_pred = train(X, y, model)
    
    print('Output results...\n')
    display_results(y_test, y_pred, cols)
    
    print('Saving model...\n')
    export_model(model)

    print('Trained model saved!')

if __name__ == '__main__':
    main()

Loading data...

Building model...

Building test and train data, training model...

Output results...

0

Report for the column (related):

Precision: 0.9
Recall: 0.89
F-score: 0.86
1

Report for the column (request):

Precision: 1.0
Recall: 1.0
F-score: 1.0
2

Report for the column (offer):

Precision: 1.0
Recall: 1.0
F-score: 1.0
3

Report for the column (aid_related):

Precision: 0.83
Recall: 0.78
F-score: 0.7
4

Report for the column (medical_help):

Precision: 0.95
Recall: 0.94
F-score: 0.94
5

Report for the column (medical_products):

Precision: 0.9
Recall: 0.89
F-score: 0.86
6

Report for the column (search_and_rescue):

Precision: 1.0
Recall: 1.0
F-score: 1.0
7

Report for the column (security):

Precision: 1.0
Recall: 1.0
F-score: 1.0
8

Report for the column (military):

Precision: 1.0
Recall: 0.97
F-score: 0.99
9

Report for the column (child_alone):

Precision: 0.9
Recall: 0.89
F-score: 0.86
10

Report for the column (water):

Precision: 0.92
Recall: 0.92
F-score: 0.89
11

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Trained model saved!


In [ ]:
mmm = build_model()
train(X, y, mmm)

In [ ]:
def run_pipeline(data_file):
    X, y = load_data(data_file)  # run ETL pipeline
    model = build_model()  # build model pipeline
    model = train(X, y, model)  # train model pipeline
    export_model(model)  # save model

In [ ]:
if __name__ == '__main__':
    data_file = sys.argv[1]  # get filename of dataset
    run_pipeline(data_file)  # run data pipeline

In [30]:
from sklearn.feature_extraction.text import CountVectorizer
    # initialize count vectorizer object
vect = CountVectorizer(tokenizer=tokenize)
    # get counts of each token (word) in text data
X = vect.fit_transform(X)
    # convert sparse matrix to numpy array to view
X.toarray()
    # view token vocabulary and counts
vect.vocabulary

In [31]:
from sklearn.feature_extraction.text import TfidfTransformer

# initialize tf-idf transformer object. Set smooth_idf parameter to false.
transformer = TfidfTransformer(smooth_idf=False)
# use counts from count vectorizer results to compute tf-idf values using the fit_transform method
tfidf = transformer.fit_transform(X)
# convert sparse matrix to numpy array to view
# you can see that the counts are normalized
tfidf.toarray()

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

# initialize tf-idf vectorizer object
vectorizer = TfidfVectorizer()

# compute bag of word counts and tf-idf values
X = vectorizer.fit_transform(X)

# convert sparse matrix to numpy array to view
X = X.toarray()

In [33]:
def model_pipeline():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            # ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', RandomForestClassifier())
    ])

    return pipeline
